In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import glob
from requests import RequestException
import csv
import re
import json
import copy
import ast
import operator
import math
import time
import zipfile as zf
from collections import Counter
from sklearn.ensemble import RandomForestRegressor
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

# Only countries that are in clusters will be kept

In order for the clustering algorithm to run, clustered countries have indicators across all 3 categories. Some of the indicators missing were computed using linear imputation through scikit by the original author of the dataset, Aljaz Kuncic

In [2]:
df = pd.read_csv('../Data/1_Institutional_Quality/institutional_quality_dataset.csv', encoding='latin-1')
df

,wbcode,country,year,legal_rel,political_rel,economic_rel,legalWIQR,politicalWIQR,economicWIQR,legalWIQR2,politicalWIQR2,economicWIQR2,legal_abs,political_abs,economic_abs,cluster
0,ADO,Andorra,1990,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ADO,Andorra,1991,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ADO,Andorra,1992,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ADO,Andorra,1993,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ADO,Andorra,1994,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4132,ZWE,Zimbabwe,2006,-2.005091,-1.549780,-2.869746,84.0,86.0,84.0,109.0,122.0,108.0,0.250514,0.253429,0.178055,1.0
4133,ZWE,Zimbabwe,2007,-2.054029,-1.595189,-2.927438,84.0,86.0,84.0,111.0,124.0,110.0,0.248133,0.241630,0.176746,1.0
4134,ZWE,Zimbabwe,2008,-2.048365,-1.634503,-2.846811,84.0,86.0,84.0,110.0,126.0,109.0,0.258128,0.225858,0.185948,1.0
4135,ZWE,Zimbabwe,2009,-1.963518,-1.290693,-2.647618,84.0,83.0,84.0,113.0,121.0,112.0,0.269821,0.298969,0.229839,1.0


In [3]:
new_df = df.loc[(df['year'] >= 2002)]
new_df

,wbcode,country,year,legal_rel,political_rel,economic_rel,legalWIQR,politicalWIQR,economicWIQR,legalWIQR2,politicalWIQR2,economicWIQR2,legal_abs,political_abs,economic_abs,cluster
12,ADO,Andorra,2002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.936833,NaN,0.815572,NaN
13,ADO,Andorra,2003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.922144,NaN,0.779470,NaN
14,ADO,Andorra,2004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.932847,NaN,0.778124,NaN
15,ADO,Andorra,2005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.919017,NaN,0.775176,NaN
16,ADO,Andorra,2006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.912810,NaN,0.774640,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4132,ZWE,Zimbabwe,2006,-2.005091,-1.549780,-2.869746,84.0,86.0,84.0,109.0,122.0,108.0,0.250514,0.253429,0.178055,1.0
4133,ZWE,Zimbabwe,2007,-2.054029,-1.595189,-2.927438,84.0,86.0,84.0,111.0,124.0,110.0,0.248133,0.241630,0.176746,1.0
4134,ZWE,Zimbabwe,2008,-2.048365,-1.634503,-2.846811,84.0,86.0,84.0,110.0,126.0,109.0,0.258128,0.225858,0.185948,1.0
4135,ZWE,Zimbabwe,2009,-1.963518,-1.290693,-2.647618,84.0,83.0,84.0,113.0,121.0,112.0,0.269821,0.298969,0.229839,1.0


In [4]:
df_nonan = new_df[new_df['cluster'].notna()]

In [5]:
df_final = df_nonan[['wbcode', 'country', 'year','legal_abs', 'political_abs', 'economic_abs','cluster']]

In [6]:
df_final

,wbcode,country,year,legal_abs,political_abs,economic_abs,cluster
54,AGO,Angola,2002,0.412211,0.281626,0.393893,1.0
55,AGO,Angola,2003,0.428805,0.297034,0.430300,1.0
56,AGO,Angola,2004,0.425183,0.285338,0.361373,1.0
57,AGO,Angola,2005,0.387639,0.296076,0.311552,1.0
58,AGO,Angola,2006,0.386354,0.321154,0.384535,1.0
...,...,...,...,...,...,...,...
4132,ZWE,Zimbabwe,2006,0.250514,0.253429,0.178055,1.0
4133,ZWE,Zimbabwe,2007,0.248133,0.241630,0.176746,1.0
4134,ZWE,Zimbabwe,2008,0.258128,0.225858,0.185948,1.0
4135,ZWE,Zimbabwe,2009,0.269821,0.298969,0.229839,1.0


In [7]:
df_final.to_csv('../Data/1_Institutional_Quality/IQs_of_only_clustered_countries.csv', index=False)